## Project: Resale Flat Data Consolidation & Cleaning 🏡
The goal here is to take the data from multiple API endpoints, load it into a single pandas DataFrame, 
and perform essential data cleaning and transformation steps. We will follow a structured process, using df.head() at each stage to inspect the data and confirm the changes.

Setup and Data Loading
This section contains the updated code to fetch all datasets and combine them into one large DataFrame. 
The fetch_dataset function has been modified to return a DataFrame instead of writing to a CSV file. The main 
function now collects all these DataFrames and uses pd.concat to merge them.

## 1. Data Fetching and Combination
This cell sets up the environment, defines the dataset sources, and fetches all HDB resale flat datasets from data.gov.sg. It combines them into a single raw DataFrame (`df_raw`) for further cleaning and analysis.

In [1]:
import nest_asyncio
import asyncio
import aiohttp
import pandas as pd
import math

# Apply nest_asyncio to allow asyncio.run() in nested environments like Jupyter Notebook
nest_asyncio.apply()

# --- Configuration ---
# Use the full set of datasets to get the complete time range from 1990 onwards
datasets = {
    "d_ebc5ab87086db484f88045b47411ebc5": "Resale Flat Prices (Based on Approval Date), 1990 - 1999",
    "d_43f493c6c50d54243cc1eab0df142d6a": "Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012",
    "d_ea9ed51da2787afaf8e51f827c304208": "Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016",
    "d_2d5ff9ea31397b66239f245f57751537": "Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014",
    "d_8b84c4ee58e3cfc0ece0d773c8ca6abc": "Resale Flat Prices Based on Registration Date from Jan-2017 onwards",
}

base_url = "https://data.gov.sg/api/action/datastore_search?resource_id="
LIMIT = 10000

# --- Asynchronous Fetching Functions (Unchanged) ---
async def fetch(session, url):
    async with session.get(url) as resp:
        return await resp.json()

async def fetch_page(session, dataset_id, offset):
    url = f"{base_url}{dataset_id}&limit={LIMIT}&offset={offset}"
    return await fetch(session, url)

# --- Main Data Retrieval Logic (Modified) ---
async def fetch_dataset(session, dataset_id, name):
    print(f"\nFetching {name}...")
    first_page = await fetch_page(session, dataset_id, 0)
    
    if not first_page.get("success"):
        print(f"Failed to fetch data for {name}.")
        return pd.DataFrame() # Return empty DataFrame on failure

    total_rows = first_page["result"]["total"]
    print(f"✅ {name}: {total_rows} total rows available.")

    total_pages = math.ceil(total_rows / LIMIT)
    offsets = [i * LIMIT for i in range(total_pages)]

    tasks = [fetch_page(session, dataset_id, offset) for offset in offsets]
    results = await asyncio.gather(*tasks)

    all_records = []
    for data in results:
        if data.get("success"):
            all_records.extend(data['result']['records'])
    
    # Return the data as a DataFrame
    return pd.DataFrame(all_records)

async def main():
    """Main function to fetch all datasets and combine them."""
    all_dfs = []
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_dataset(session, dataset_id, name) for dataset_id, name in datasets.items()]
        results = await asyncio.gather(*tasks)
        for df in results:
            if not df.empty:
                all_dfs.append(df)
            else:
                print("‼️ An entire dataset was not loaded due to a fetch error.")
    
    # Combine all DataFrames into one and return
    if all_dfs:
        return pd.concat(all_dfs, ignore_index=True)
    return pd.DataFrame()

# Execute the main function to get the combined raw DataFrame
df_raw = asyncio.run(main())

# Remove '_id' column
if '_id' in df_raw.columns:
    df_raw = df_raw.drop(columns=['_id'])

print("\n--- Initial Raw DataFrame ---")
print(df_raw.head())



Fetching Resale Flat Prices (Based on Approval Date), 1990 - 1999...

Fetching Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012...

Fetching Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016...

Fetching Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014...

Fetching Resale Flat Prices Based on Registration Date from Jan-2017 onwards...
✅ Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014: 52203 total rows available.
✅ Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012: 369651 total rows available.
✅ Resale Flat Prices (Based on Approval Date), 1990 - 1999: 287196 total rows available.
✅ Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016: 37153 total rows available.
✅ Resale Flat Prices Based on Registration Date from Jan-2017 onwards: 214331 total rows available.

--- Initial Raw DataFrame ---
     month        town flat_type block       street_name storey_range  \
0

# Print the unique values in the raw month column to inspect for inconsistencies.
print(df_raw['month'].unique())

num_years = df['year'].unique()
print(f'There are {num_years} unique years in the dataframe.')

## 2. Data Cleaning
This cell performs essential cleaning and transformation on the raw DataFrame. It standardizes column names, removes duplicates, formats columns, computes derived features (like `remaining_lease_years`), and prepares the data for analytics and modeling.

# --- Data Cleaning ---
import numpy as np
from datetime import datetime
df = df_raw.copy()

# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Rename _id to id if present
if '_id' in df.columns:
    df = df.rename(columns={'_id': 'id'})

# Remove duplicate rows if id exists
if 'id' in df.columns:
    df = df.drop_duplicates(subset='id')

# Clean string columns
for col in ['block', 'street_name', 'town', 'flat_type', 'flat_model']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.upper()

# Convert resale_price and floor_area_sqm to numeric
for col in ['resale_price', 'floor_area_sqm']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Format month column to YYYY-MM and extract year
if 'month' in df.columns:
    df['month'] = pd.to_datetime(df['month'], format='%Y-%m', errors='coerce').dt.strftime('%Y-%m')
    df['year'] = pd.to_datetime(df['month'], format='%Y-%m', errors='coerce').dt.year

# Lease commence year to numeric
if 'lease_commence_date' in df.columns:
    df['lease_commence_date'] = pd.to_numeric(df['lease_commence_date'], errors='coerce')



# Parse storey_range into min/max
if 'storey_range' in df.columns:
    try:
        df[['storey_min', 'storey_max']] = df['storey_range'].str.split(' TO ', expand=True).astype(float)
    except Exception:
        pass


"""
# Compute remaining_lease_years as 'YY years MM months' at present date
def format_remaining_lease_years(row):
    lease_year = row.get('lease_commence_date')
    if pd.isna(lease_year) or not lease_year:
        return None
    try:
        lease_year = int(lease_year)
    except Exception:
        return None
    today = datetime.now()
    years_elapsed = today.year - lease_year
    months_elapsed = today.month - 1
    years_left = 99 - years_elapsed
    months_left = 0 - months_elapsed
    if months_left < 0:
        years_left -= 1
        months_left += 12
    if years_left < 0:
        return '0 years 0 months'
    return f'{years_left} years {months_left} months'

df['remaining_lease_years'] = df.apply(format_remaining_lease_years, axis=1)
"""



# Remove the old remaining_lease column if present
if 'remaining_lease' in df.columns:
    df = df.drop(columns=['remaining_lease'])

# Calculate price_per_sqm and round to 2 decimals
if 'resale_price' in df.columns and 'floor_area_sqm' in df.columns:
    df['price_per_sqm'] = (df['resale_price'] / df['floor_area_sqm']).round(2)

"""
# Drop outliers in resale_price
if 'resale_price' in df.columns:
    df = df[(df['resale_price'] > 50000) & (df['resale_price'] < 2000000)]
"""

print("\n--- Cleaned DataFrame ---")
print(df.head())

In [2]:
# --- Data Cleaning ---
import numpy as np
import pandas as pd
from datetime import datetime

df = df_raw.copy()

# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Clean string columns
for col in ['block', 'street_name', 'town', 'flat_type', 'flat_model']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.upper()

# Convert resale_price and floor_area_sqm to numeric
for col in ['resale_price', 'floor_area_sqm']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Format month column to YYYY-MM and extract year
if 'month' in df.columns:
    df['month'] = pd.to_datetime(df['month'], format='%Y-%m', errors='coerce').dt.strftime('%Y-%m')
    df['year'] = pd.to_datetime(df['month'], format='%Y-%m', errors='coerce').dt.year

# Lease commence year to numeric
if 'lease_commence_date' in df.columns:
    df['lease_commence_date'] = pd.to_numeric(df['lease_commence_date'], errors='coerce')

# --- Compute remaining lease at point of sale ---
def format_remaining_lease_at_sale(row):
    lease_year = row.get('lease_commence_date')
    sale_month = row.get('month')  # should be YYYY-MM string
    if pd.isna(lease_year) or not lease_year:
        return None
    if pd.isna(sale_month) or not sale_month:
        return None
    try:
        lease_year = int(lease_year)
        sale_dt = pd.to_datetime(sale_month, format='%Y-%m', errors='coerce')
        if pd.isna(sale_dt):
            return None
    except Exception:
        return None

    # Years and months elapsed since lease commencement
    years_elapsed = sale_dt.year - lease_year
    months_elapsed = sale_dt.month - 1
    years_left = 99 - years_elapsed
    months_left = 0 - months_elapsed
    if months_left < 0:
        years_left -= 1
        months_left += 12
    if years_left < 0:
        return '0 years 0 months'
    return f'{years_left} years {months_left} months'

df['remaining_lease_at_point_of_sale'] = df.apply(format_remaining_lease_at_sale, axis=1)

# Parse storey_range into min/max
if 'storey_range' in df.columns:
    try:
        df[['storey_min', 'storey_max']] = df['storey_range'].str.split(' TO ', expand=True).astype(float)
    except Exception:
        pass

# Remove the old remaining_lease column if present
if 'remaining_lease' in df.columns:
    df = df.drop(columns=['remaining_lease'])

# Calculate price_per_sqm and round to 2 decimals
if 'resale_price' in df.columns and 'floor_area_sqm' in df.columns:
    df['price_per_sqm'] = (df['resale_price'] / df['floor_area_sqm']).round(2)

"""
# Drop outliers in resale_price
if 'resale_price' in df.columns:
    df = df[(df['resale_price'] > 50000) & (df['resale_price'] < 2000000)]

# After all cleaning steps
df['id'] = range(1, len(df) + 1)

"""

print("\n--- Cleaned DataFrame ---")
print(df.head())



--- Cleaned DataFrame ---
     month        town flat_type block       street_name storey_range  \
0  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
1  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     04 TO 06   
2  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
3  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     07 TO 09   
4  1990-01  ANG MO KIO    3 ROOM   216  ANG MO KIO AVE 1     04 TO 06   

   floor_area_sqm      flat_model  lease_commence_date  resale_price  year  \
0            31.0        IMPROVED                 1977        9000.0  1990   
1            31.0        IMPROVED                 1977        6000.0  1990   
2            31.0        IMPROVED                 1977        8000.0  1990   
3            31.0        IMPROVED                 1977        6000.0  1990   
4            73.0  NEW GENERATION                 1976       47200.0  1990   

  remaining_lease_at_point_of_sale  storey_min  storey_max  price

In [3]:
# Print the unique values in the raw month column to inspect for inconsistencies.
print(df_raw['month'].unique())
num_years = df['year'].unique()
print(f'There are {num_years} unique years in the dataframe.')

['1990-01' '1990-02' '1990-03' '1990-04' '1990-05' '1990-06' '1990-07'
 '1990-08' '1990-09' '1990-10' '1990-11' '1990-12' '1991-01' '1991-02'
 '1991-03' '1991-04' '1991-05' '1991-06' '1991-07' '1991-08' '1991-09'
 '1991-10' '1991-11' '1991-12' '1992-01' '1992-02' '1992-03' '1992-04'
 '1992-05' '1992-06' '1992-07' '1992-08' '1992-09' '1992-10' '1992-11'
 '1992-12' '1993-01' '1993-02' '1993-03' '1993-04' '1993-05' '1993-06'
 '1993-07' '1993-08' '1993-09' '1993-10' '1993-11' '1993-12' '1994-01'
 '1994-02' '1994-03' '1994-04' '1994-05' '1994-06' '1994-07' '1994-08'
 '1994-09' '1994-10' '1994-11' '1994-12' '1995-01' '1995-02' '1995-03'
 '1995-04' '1995-05' '1995-06' '1995-07' '1995-08' '1995-09' '1995-10'
 '1995-11' '1995-12' '1996-01' '1996-02' '1996-03' '1996-04' '1996-05'
 '1996-06' '1996-07' '1996-08' '1996-09' '1996-10' '1996-11' '1996-12'
 '1997-01' '1997-02' '1997-03' '1997-04' '1997-05' '1997-06' '1997-07'
 '1997-08' '1997-09' '1997-10' '1997-11' '1997-12' '1998-01' '1998-02'
 '1998

## 3. Star Schema Construction
This cell builds a star schema from the cleaned DataFrame. It creates dimension tables (flat, location, storey, time) and a fact table for resale transactions, enabling efficient analytics and reporting.

In [4]:
import pandas as pd

# --- Build Star Schema from Cleaned DataFrame ---

# 1. Flat Dimension
flat_dim = df[['flat_type', 'flat_model', 'floor_area_sqm', 'lease_commence_date']].drop_duplicates().reset_index(drop=True)
flat_dim['flat_id'] = flat_dim.index + 1  # synthetic key
flat_dim = flat_dim[['flat_id', 'flat_type', 'flat_model', 'floor_area_sqm', 'lease_commence_date']]

# 2. Location Dimension
location_dim = df[['town', 'street_name', 'block']].drop_duplicates().reset_index(drop=True)
location_dim['location_id'] = location_dim.index + 1
location_dim = location_dim[['location_id', 'town', 'street_name', 'block']]

# 3. Storey Dimension
storey_dim = df[['storey_range', 'storey_min', 'storey_max']].drop_duplicates().reset_index(drop=True)
storey_dim['storey_id'] = storey_dim.index + 1
storey_dim = storey_dim[['storey_id', 'storey_range', 'storey_min', 'storey_max']]

# 4. Time Dimension
time_dim = df[['year', 'month']].drop_duplicates().reset_index(drop=True)
time_dim['time_id'] = time_dim.index + 1
time_dim = time_dim[['time_id', 'year', 'month']]

# 5. Fact Table (Resale Transactions)
fact_table = df.merge(flat_dim, on=['flat_type', 'flat_model', 'floor_area_sqm', 'lease_commence_date'], how='left') \
               .merge(location_dim, on=['town', 'street_name', 'block'], how='left') \
               .merge(storey_dim, on=['storey_range', 'storey_min', 'storey_max'], how='left') \
               .merge(time_dim, on=['year', 'month'], how='left')

fact_table = fact_table[['flat_id', 'location_id', 'storey_id', 'time_id',
                         'resale_price', 'price_per_sqm', 'remaining_lease_at_point_of_sale']]


# --- Final Outputs ---
print("\nFlat Dimension:\n", flat_dim.head())
print("\nLocation Dimension:\n", location_dim.head())
print("\nStorey Dimension:\n", storey_dim.head())
print("\nTime Dimension:\n", time_dim.head())
print("\nFact Table:\n", fact_table.head())



Flat Dimension:
    flat_id flat_type      flat_model  floor_area_sqm  lease_commence_date
0        1    1 ROOM        IMPROVED            31.0                 1977
1        2    3 ROOM  NEW GENERATION            73.0                 1976
2        3    3 ROOM  NEW GENERATION            67.0                 1977
3        4    3 ROOM  NEW GENERATION            82.0                 1976
4        5    3 ROOM  NEW GENERATION            67.0                 1976

Location Dimension:
    location_id        town       street_name block
0            1  ANG MO KIO  ANG MO KIO AVE 1   309
1            2  ANG MO KIO  ANG MO KIO AVE 1   216
2            3  ANG MO KIO  ANG MO KIO AVE 3   211
3            4  ANG MO KIO  ANG MO KIO AVE 3   202
4            5  ANG MO KIO  ANG MO KIO AVE 3   235

Storey Dimension:
    storey_id storey_range  storey_min  storey_max
0          1     10 TO 12        10.0        12.0
1          2     04 TO 06         4.0         6.0
2          3     07 TO 09         7.0   

In [30]:
num_years = df['year'].unique()
print(f'There are {num_years} unique years in the dataframe.')

There are [1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2015 2016 2013 2014 2017
 2018 2019 2020 2021 2022 2023 2024 2025] unique years in the dataframe.


## 4. Referential Integrity Check
This cell verifies that all fact table rows are correctly linked to their dimension tables and that transaction IDs are unique and sequential.

In [5]:
# --- Referential Integrity Check ---
missing_flat = fact_table['flat_id'].isna().sum()
missing_loc = fact_table['location_id'].isna().sum()
missing_storey = fact_table['storey_id'].isna().sum()
missing_time = fact_table['time_id'].isna().sum()

print("\n--- Referential Integrity Check ---")
print(f"Missing flat_id: {missing_flat}")
print(f"Missing location_id: {missing_loc}")
print(f"Missing storey_id: {missing_storey}")
print(f"Missing time_id: {missing_time}")

if all(x == 0 for x in [missing_flat, missing_loc, missing_storey, missing_time]):
    print("✅ All fact rows are linked correctly, transaction_id is unique and sequential.")
else:
    print("⚠️ Warning: Issues exist in fact table or dimension links.")



--- Referential Integrity Check ---
Missing flat_id: 0
Missing location_id: 0
Missing storey_id: 0
Missing time_id: 0
✅ All fact rows are linked correctly, transaction_id is unique and sequential.


## 5. Save Outputs
This cell saves all dimension tables, the fact table, and the cleaned DataFrame to CSV files in the `star_schema_outputs` directory for further use or sharing.

In [6]:
import os  # <--- make sure this is at the top of your notebook/script

# --- Save Outputs ---
output_dir = "star_schema_outputs"
os.makedirs(output_dir, exist_ok=True)

flat_dim.to_csv(os.path.join(output_dir, "flat_dim.csv"), index=False)
location_dim.to_csv(os.path.join(output_dir, "location_dim.csv"), index=False)
storey_dim.to_csv(os.path.join(output_dir, "storey_dim.csv"), index=False)
time_dim.to_csv(os.path.join(output_dir, "time_dim.csv"), index=False)
fact_table.to_csv(os.path.join(output_dir, "resale_transactions_fact.csv"), index=False)
df.to_csv(os.path.join(output_dir, "cleaned_dataframe.csv"), index=False)

print("\n✅ All tables saved to:", output_dir)


✅ All tables saved to: star_schema_outputs


## 6. CREATES TABLES IN POSTGRES
User to update PGadmin username and password in the cell below.

In [8]:
# --- Load All DataFrames to PostgreSQL ---
from sqlalchemy import create_engine
import sqlalchemy
!pip install psycopg2-binary
 
# --- User must set these values for their PostgreSQL instance ---
PG_USER = 'postgres'      # e.g. 'postgres'
PG_PASSWORD = 'Anypass78'  # e.g. 'mypassword'
PG_HOST = 'localhost'          # e.g. 'localhost' or IP address
PG_PORT = '5432'               # default PostgreSQL port
db_name = 'resale_flat_sg2'
 
# --- Create the database if it doesn't exist ---
default_engine = create_engine(f'postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/postgres')
with default_engine.connect() as conn:
    conn.execute(sqlalchemy.text(f'COMMIT'))
    # Check if database exists before trying to create it
    result = conn.execute(sqlalchemy.text(f"SELECT 1 FROM pg_database WHERE datname = '{db_name}'"))
    exists = result.fetchone()
    if not exists:
        conn.execute(sqlalchemy.text(f'CREATE DATABASE {db_name}'))
        print(f"Database '{db_name}' created.")
    else:
        print(f"Database '{db_name}' already exists. Skipping creation.")
 
# --- Connect to the new database ---
engine = create_engine(f'postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{db_name}')
 
# --- Write all tables ---
flat_dim.to_sql('flat_dim', engine, if_exists='replace', index=False)
location_dim.to_sql('location_dim', engine, if_exists='replace', index=False)
storey_dim.to_sql('storey_dim', engine, if_exists='replace', index=False)
time_dim.to_sql('time_dim', engine, if_exists='replace', index=False)
fact_table.to_sql('resale_transactions_fact', engine, if_exists='replace', index=False)
 
print('✅ All tables loaded to PostgreSQL database resale_flat_sg.')

Database 'resale_flat_sg2' created.
✅ All tables loaded to PostgreSQL database resale_flat_sg.
